In [1]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

DEBUG:__main__:hello


In [2]:
# ipynb形式のライブラリノートを.py形式に変更したものをインポート
import lib
import lib.lab_lib
from lib.lab_lib import *

DEBUG:lib.lab_lib:hello


In [3]:
benchmark_name = "cg"

classes = ["A", "B", "C", "D", "E", "F"]
processes = [2, 4, 8, 16, 32, 64, 128, 256]

dict_column_names_JP = {
    "functionName": "関数名",
    "modelLin": "線形",
    "modelIp": "反比例",
    "modelLog": "対数",
    "modelLinAndIp": "線形＆反比例",
    "modelLinAndLog": "線形＆対数",
    "modelIpAndLin": "反比例＆線形",
    "modelIpAndLog": "反比例＆対数",
    "modelLogAndLin": "対数＆線形",
    "modelLogAndIp": "対数＆反比例",
}

model_names_list_JP = list(dict_column_names_JP.values())
model_names_list_JP.remove("関数名")

model_names_list = list(dict_column_names_JP.keys())
model_names_list.remove("functionName")

date = "2022年2月2日"

# コア数と問題サイズの両方を振る

rawDF = return_rawDF_with_init_param(
    benchmark_name=benchmark_name,
    classes=classes,
    processes=processes,
    csv_dir_path="./csv_files/",
)

exp_var = rawDF.columns.tolist()
for element_be_removed in [
    "functionName",
    "functionCallNum",
    "intBenchmarkClass",
    "benchmarkName",
    "benchmarkClass",
]:
    exp_var.remove(element_be_removed)
res_var = ["functionCallNum"]

rawDF_CONJ_GRAD = rawDF[rawDF["functionName"] == "CONJ_GRAD"]
rawDF_CONJ_GRAD

# returned_MAPE_table = return_MAPE_Table_DF_from_rawDF(
#     rawDF=rawDF,
#     exp_var_list=exp_var,
#     res_var_list=res_var,
#     model_name_list=model_names_list,
# )
# df = returned_MAPE_table.rename(columns=dict_column_names_JP)
# df = df.set_index("関数名")
# df = df.sort_index(axis=0)
# df = df.sort_index(axis=1)

# df = addLowestMAPEsModelNameColumn(df, version=2, model_name_list=model_names_list_JP)

# print(
#     df.to_latex(
#         caption=f"ベンチマークプログラム{benchmark_name.upper()}における各モデルでのMAPE",
#         label=f"{date}everyModelsMAPEtableIn{benchmark_name.upper()}",
#     )
# )

,functionName,functionCallNum,benchmarkName,benchmarkClass,process,intBenchmarkClass,na,nonzer,niter,shift
5,CONJ_GRAD,16.0,cg,A,2,1,14000,11,15,20
5,CONJ_GRAD,16.0,cg,A,4,1,14000,11,15,20
5,CONJ_GRAD,16.0,cg,A,8,1,14000,11,15,20
5,CONJ_GRAD,16.0,cg,A,16,1,14000,11,15,20
5,CONJ_GRAD,16.0,cg,A,32,1,14000,11,15,20
5,CONJ_GRAD,16.0,cg,A,64,1,14000,11,15,20
5,CONJ_GRAD,16.0,cg,A,128,1,14000,11,15,20
6,CONJ_GRAD,16.0,cg,A,256,1,14000,11,15,20
2,CONJ_GRAD,31.0,cg,B,2,4,75000,13,75,60
2,CONJ_GRAD,46.0,cg,B,4,4,75000,13,75,60


# （2022年2月23日～）

# 新モデルの実装

In [4]:
# TODO:新モデルに変更する


class Model_ProcessesDevidedByProblemSize_ForMultipleRegression(
    ModelBaseForMultipleRegression
):
    """プロセス数を問題サイズの変数群の線形和で割ったモデル

    組み合わせモデルを実現するためのクラス

    Attributes:
        equationDict (dict): キー・バリューが列名・変形モデル（線形、反比例、対数など）
        lr : モデルのオブジェクト
        dataXForPredict : 説明変数のDF
        dataTForPredict : 目的変数のDF
    Note:
        モデルの式は次の通り
        関数コール回数 = (プロセス数)/(a * 問題サイズ1 + b * 問題サイズ2 + ... + n * 問題サイズn + c)
    """


    
    def build_model(self) -> bool:
        """build_model(self)

        inputDFのデータからモデル構築する。

        Args:
            self : none

        Returns: boolean。成功ならTrue,失敗ならFalse
        """
        
        self.len_list_coefficient :int = 5
        
        # TODO:curve_fit()

        # list_exp:説明変数のリスト
        # list_expをlist_inputとして利用
        list_exp :np.ndarray = self.rawExplanaoryVariable.to_numpy().T
        # list_res:目的変数のリスト
        list_res :np.ndarray = self.rawResponseVariable.to_numpy()
        # list_resが複数の列を持っていると予測ができるか不確かなため、警告を出す
        if list_res.shape[1] != 1:
            warnings.warn("目的変数の個数が想定と異なります")
            return False
        list_res = list_res.ravel()
        
        # 説明変数(=len(list_exp))の個数が{self.len_list_coefficient}より大きい(={self.len_list_coefficient}+1以上)なら警告を出す
        if len(list_exp) > self.len_list_coefficient:
            warnings.warn(f"説明変数の個数が想定({self.len_list_coefficient})より大きいです（len(list_exp)={len(list_exp)}）")
            return False
        # list_exp(=list_input) の0埋めをする
        if list_exp.shape[0] < self.len_list_coefficient:
            while( len(list_exp) != self.len_list_coefficient ):
                list_exp = np.append(list_exp, [np.zeros(list_exp.shape[1])], axis=0)


        # モデルの構築
        popt: np.ndarray
        pcov :np.ndarray

        print(f"list_exp.shape = {list_exp.shape}")
        print(f"list_res.shape = {list_res.shape}")

        tmp_result = processesDevidedByProblemSize(list_input=list_exp, coefficient_a=11, coefficient_b=13, coefficient_c=17, coefficient_d=19, coefficient_e=23, intercept=60)
        print(f"tmp_result[processesDevidedByProblemSize] = {tmp_result}")

        print(f"list_res = {list_res}")
        

        self.dataXForPredict :np.ndarray = list_exp
        self.dataTForPredict :np.ndarray = list_res

        self.popt, self.pcov = curve_fit(processesDevidedByProblemSize, list_exp, list_res)

        return True

    def predict(self, inputDF) -> np.ndarray:
        """predict(self, inputDF)

        inputDFのデータから構築されたモデルを使って予測を行う

        Args:
            self : none
            inputDF (pandas.DataFrame): 構築されたモデルを用いて予測に使うDF

        Returns:
            pandas.DataFrame: 構築されたモデルから予測された値。型に確証なし
        """
        
        # TODO:predict()を必要に応じて実装する
        

    def returnMAPE(self) -> float:
        """calcMAPE(self)

        モデルの構築に使用されたデータからMAPEを算出する

        Args:
            self : none

        Returns:
            list: モデルの構築に用いたデータから予測された値
            int: 失敗した場合、-1
        """

        return -1.0
        # TODO:returnMAPE()を必要に応じて実装する
        
# めも：list_input と coefficient_* の個数は同じにする。できない場合は0埋めをしたのを数合わせにlist_inputに入れる
# モデル式の宣言
def processesDevidedByProblemSize(list_input :list[np.ndarray] = [], coefficient_a :float = 0, coefficient_b :float = 0, coefficient_c :float = 0, coefficient_d :float = 0, coefficient_e :float = 0, intercept :float = 0) -> list[float]:
    """processesDevidedByProblemSize(list_input = [], coefficient_a = 0, coefficient_b = 0, coefficient_c = 0, coefficient_d = 0, coefficient_e = 0, intercept = 0)

    inputDFのデータから構築されたモデルを使って予測を行う

    Args:
        list_input : 変数の入った行列。現状は一般の行列と異なり [[<列データ1>], [<列データ2>], ... , [<列データn>]]の形式
        coefficient_a : 係数1
        coefficient_b : 係数2
        coefficient_c : 係数3
        coefficient_d : 係数4
        coefficient_e : 係数5
        intercept : 切片

    Returns:
        np.ndarray[float] : 計算された値。
    """
    list_coefficient :list[float] = [coefficient_a, coefficient_b, coefficient_c, coefficient_d, coefficient_e]
    # list_inputの要素数と有効なa,b,c,d,eの個数が同じことを確認
    if len(list_input) != len(list_coefficient):
        warnings.warn(f"len(list_input){len(list_input)} != len(list_coefficient){len(list_coefficient)}")
    
    result :list[float] = []
    for elm_a, elm_b, elm_c, elm_d, elm_e in zip(list_input[0], list_input[1], list_input[2], list_input[3], list_input[4]):
        result.append( ( coefficient_a * elm_a) / (coefficient_b * elm_b + coefficient_c * elm_c + coefficient_d * elm_d + coefficient_e * elm_e) + intercept )

    return result

def test_processesDevidedByProblemSize():
    """test_processesDevidedByProblemSize()

    processesDevidedByProblemSizeのテスト
    """

    list_A :list[int] = [1,2,3,4]
    list_B :list[int] = 10 * list_A
    list_C :list[int] = 100 * list_A

    a : int = 5
    b : int = 6
    c : int = 7
    d : int = 8

    list_T_expect :list[int] = []
    for i in range(len(list_A)):
        list_T_expect.append((a * list_A[i])/(b*list_B[i] + c * list_C[i]) + d)

    list_input_for_actually :list[np.ndarray] = [np.array(list_A), np.array(list_B), np.array(list_C), np.zeros(len(list_A)), np.zeros(len(list_A))]
    list_T_actually :np.ndarray = processesDevidedByProblemSize(list_input=list_input_for_actually, coefficient_a=a, coefficient_b=b, coefficient_c=c, intercept=d)

    assert (list_T_expect == list_T_actually)

def test_Model_ProcessesDevidedByProblemSize_ForMultipleRegression():
    """test_ModelMultipleEquationForMultipleRegression()

    ModelMultipleEquationForMultipleRegressionのテスト
    """

    # 説明変数
    plotX = np.linspace(10, 20, 11)
    plotY = np.linspace(20, 30, 11)
    plotZ = np.linspace(30, 40, 11)
    # 目的変数
    a = 11
    b = 13
    c = 17
    d = 19
    plotT = (a * plotX) / (b * plotY + c * plotZ) + d

    # DFを作成する
    # カラム名のリスト
    columnNames = ["plotX", "plotY", "plotZ", "plotT"]
    datumForDF = [plotX, plotY, plotZ, plotT]
    inputDFForTest = pd.DataFrame(index=columnNames, data=datumForDF).T
    inputDFForTest["functionName"] = "functionName"

    # 目的変数・説明変数のカラム名のリスト
    # 目的変数のカラム名のリスト
    columnNamesForExp = ["plotX", "plotY", "plotZ"]
    # 説明変数のカラム名のリスト
    columnNamesForRes = ["plotT"]

    # 予測をする
    # モデルオブジェクトの作成
    objectModel = Model_ProcessesDevidedByProblemSize_ForMultipleRegression(
        inputDF=inputDFForTest,
        explanatoryVariableColumnNames=columnNamesForExp,
        responseVariableColumnNames=columnNamesForRes,
        conditionDictForTest={},
    )
    # TODO:モデルの構築
    objectModel.build_model()
    # モデル構築に用いたデータとのMAPEによって実装がうまくいっているかどうかの判定を行う
    MAPE = objectModel.returnMAPE()
    print(f"MAPE = {MAPE}")
    assert 0< MAPE < 1

In [5]:
test_processesDevidedByProblemSize()

In [6]:
test_Model_ProcessesDevidedByProblemSize_ForMultipleRegression()

list_exp.shape = (5, 11)
list_res.shape = (11,)
tmp_result[processesDevidedByProblemSize] = [60.142857142857146, 60.15125, 60.159036144578316, 60.16627906976744, 60.173033707865166, 60.17934782608695, 60.18526315789474, 60.190816326530616, 60.19603960396039, 60.20096153846154, 60.205607476635514]
list_res = [19.14285714 19.15125    19.15903614 19.16627907 19.17303371 19.17934783
 19.18526316 19.19081633 19.1960396  19.20096154 19.20560748]
MAPE = -1.0


/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


AssertionError: 

# （2022年2月22日～）【個人作業】

# 回帰木による予測

In [ ]:
from sklearn.tree import DecisionTreeRegressor

rawDF

In [ ]:
rawDF_CONJ_GRAD = rawDF[rawDF["functionName"] == "CONJ_GRAD"]
rawDF_CONJ_GRAD

In [ ]:
print(f"exp_var = {exp_var}")

print(f"res_var = {res_var}")

print(f"rawDF_CONJ_GRAD.shape = {rawDF_CONJ_GRAD.shape}")

In [ ]:
X = rawDF_CONJ_GRAD.filter(items=exp_var)
Y = rawDF_CONJ_GRAD.filter(items=res_var)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.1, random_state=1234
)

# X_train, X_test, Y_train, Y_test

In [ ]:
reg = DecisionTreeRegressor(max_leaf_nodes=20)

model = reg.fit(X_train, Y_train)
print(f"model = {model}")

In [ ]:
from sklearn.metrics import r2_score

YHat = model.predict(X_test)
r2 = r2_score(Y_test, YHat)
print("R^2 = ", r2)

In [ ]:
from sklearn import tree
from IPython.display import Image
import pydotplus

dot_data = tree.export_graphviz(
    model,
    out_file=None,
    feature_names=rawDF_CONJ_GRAD.feature_names,
    class_names="functionCallNum",
    filled=True,
)
graph = pydotplus.graph_from_dot_data(dot_data)

Image(graph.create_png())